In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/Kaggle/Kaggle/Scripts/')
import my_utils

In [ ]:
import pandas as pd
import numpy as np
import warnings
from my_utils import remove_miss_columns
warnings.filterwarnings('ignore')
from sklearn.preprocessing import FunctionTransformer

In [9]:
final_train = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/ctrain.csv')
final_test = pd.read_csv('/content/drive/Shareddrives/Kaggle/Kaggle/Data/ctest.csv')

In [30]:
target = 'contest-tmp2m-14d__tmp2m'
X_train = final_train.drop([target], axis=1)
X_test = final_test.drop([target], axis=1)

features = X_train.columns
y_train = final_train[target]

In [28]:
final_train.shape

(337256, 207)

### LightLGBM cross validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
import lightgbm as lgbm
from tqdm import tqdm

k = 5
kfold = KFold(k,shuffle=True, random_state=42)
val_scores = []
test_preds= []

for i,(train_idxs,val_idxs) in tqdm(enumerate(kfold.split(final_train))):

    X_train = final_train.iloc[train_idxs][features]
    y_train = final_train.iloc[train_idxs][target]
    X_val = final_train.iloc[val_idxs][features]
    y_val = final_train.iloc[val_idxs][target]
    test = final_train.iloc[val_idxs]
    
    params= {
     'learning_rate':0.02,
     'lambda_l1': 1.945,
     'num_leaves': 87,
     'feature_fraction': 0.79,
     'bagging_fraction': 0.93,
     'bagging_freq': 4,
     'min_data_in_leaf': 103,
     'max_depth': 17,
     'num_iterations':5000
    }
    
    model = lgbm.LGBMRegressor(**params)    

    model.fit(X= X_train,
              y= y_train,
              eval_set = (X_val,y_val),
              early_stopping_rounds = 100,
              verbose=500
             )
    preds = model.predict(X_val)
    rmse = mse(y_val, preds,squared=False)
    val_scores.append(rmse)
    print(f'=== Fold {i} RMSE {rmse} ====')
    
    preds = model.predict(test[features])
    test_preds.append(preds)
    
print(f'=== Average RMSE of {k} Folds: {np.mean(val_scores)} ====')

0it [00:00, ?it/s]

Training until validation scores don't improve for 100 rounds.
[500]	valid_0's l2: 0.603344
[1000]	valid_0's l2: 0.328388
[1500]	valid_0's l2: 0.232018
[2000]	valid_0's l2: 0.181182
[2500]	valid_0's l2: 0.149465
[3000]	valid_0's l2: 0.127644
[3500]	valid_0's l2: 0.111852
[4000]	valid_0's l2: 0.0999349
[4500]	valid_0's l2: 0.0910619
[5000]	valid_0's l2: 0.0835539
Did not meet early stopping. Best iteration is:
[5000]	valid_0's l2: 0.0835539
=== Fold 0 RMSE 0.28905699229668863 ====


1it [23:13, 1393.31s/it]

Training until validation scores don't improve for 100 rounds.
[500]	valid_0's l2: 0.613532
[1000]	valid_0's l2: 0.336144
[1500]	valid_0's l2: 0.238272
[2000]	valid_0's l2: 0.186425
[2500]	valid_0's l2: 0.152156
[3000]	valid_0's l2: 0.130113
[3500]	valid_0's l2: 0.114159
[4000]	valid_0's l2: 0.101829
[4500]	valid_0's l2: 0.092235
[5000]	valid_0's l2: 0.0845459
Did not meet early stopping. Best iteration is:
[5000]	valid_0's l2: 0.0845459
=== Fold 1 RMSE 0.29076785097063923 ====


2it [46:30, 1395.59s/it]

...

Training until validation scores don't improve for 100 rounds.
[500]	valid_0's l2: 0.622913
[1000]	valid_0's l2: 0.333782
[1500]	valid_0's l2: 0.237192
[2000]	valid_0's l2: 0.185364
[2500]	valid_0's l2: 0.152128
[3000]	valid_0's l2: 0.130132
[3500]	valid_0's l2: 0.114069
[4000]	valid_0's l2: 0.102126
[4500]	valid_0's l2: 0.0928804
[5000]	valid_0's l2: 0.0854259
Did not meet early stopping. Best iteration is:
[5000]	valid_0's l2: 0.0854259
=== Fold 3 RMSE 0.2922771249596201 ====


4it [1:32:08, 1377.66s/it]

Training until validation scores don't improve for 100 rounds.
[500]	valid_0's l2: 0.617075
[1000]	valid_0's l2: 0.338035
[1500]	valid_0's l2: 0.240411
[2000]	valid_0's l2: 0.187812
[2500]	valid_0's l2: 0.155226
[3000]	valid_0's l2: 0.132158
[3500]	valid_0's l2: 0.116079
[4000]	valid_0's l2: 0.103777
[4500]	valid_0's l2: 0.0939991
[5000]	valid_0's l2: 0.086092
Did not meet early stopping. Best iteration is:
[5000]	valid_0's l2: 0.086092
=== Fold 4 RMSE 0.2934143349257904 ====


5it [1:55:03, 1380.66s/it]

=== Average RMSE of 5 Folds: 0.29145169045475383 ====


### Catboost cross validation

In [13]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.2 MB/s eta 0:00:00


In [ ]:
import catboost
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm

k = 5
kfold = KFold(k,shuffle=True, random_state=123)
val_scores = []
test_preds= []

for i,(train_idxs,val_idxs) in tqdm(enumerate(kfold.split(final_train))):

    X_train = final_train.iloc[train_idxs][features]
    y_train = final_train.iloc[train_idxs][target]
    X_val = final_train.iloc[val_idxs][features]
    y_val = final_train.iloc[val_idxs][target]
    test = final_train.iloc[val_idxs]
    
    params = {
        # 'task_type': 'GPU',
        'iterations': 5000,
        'learning_rate': 0.2,
        'depth': 10
    }
    
    model = CatBoostRegressor(**params)   
    model.fit(X= X_train,
              y= y_train,
              eval_set = (X_val,y_val),
              early_stopping_rounds = 100,
              verbose=500
             ) 
    preds = model.predict(X_val)
    rmse = mse(y_val, preds,squared=False)
    val_scores.append(rmse)
    print(f'=== Fold {i} RMSE {rmse} ====')
    
    preds = model.predict(test[features])
    test_preds.append(preds)
    
print(f'=== Average RMSE of {k} Folds: {np.mean(val_scores)} ====')


0it [00:00, ?it/s]

0:	learn: 8.2543412	test: 8.2365913	best: 8.2365913 (0)	total: 982ms	remaining: 1h 21m 51s
500:	learn: 0.3621359	test: 0.4121561	best: 0.4121561 (500)	total: 8m 22s	remaining: 1h 15m 10s
1000:	learn: 0.2427853	test: 0.3114966	best: 0.3114966 (1000)	total: 16m 34s	remaining: 1h 6m 13s
1500:	learn: 0.1882278	test: 0.2697206	best: 0.2697206 (1500)	total: 24m 46s	remaining: 57m 45s
2000:	learn: 0.1556973	test: 0.2471657	best: 0.2471657 (2000)	total: 32m 52s	remaining: 49m 16s
2500:	learn: 0.1331839	test: 0.2327937	best: 0.2327937 (2500)	total: 41m 38s	remaining: 41m 36s
3000:	learn: 0.1168519	test: 0.2232287	best: 0.2232287 (3000)	total: 49m 43s	remaining: 33m 7s
3500:	learn: 0.1040210	test: 0.2161476	best: 0.2161476 (3500)	total: 57m 57s	remaining: 24m 48s
4000:	learn: 0.0938765	test: 0.2109719	best: 0.2109719 (4000)	total: 1h 6m 13s	remaining: 16m 32s
4500:	learn: 0.0852424	test: 0.2067907	best: 0.2067907 (4500)	total: 1h 14m 25s	remaining: 8m 15s
4999:	learn: 0.0780956	test: 0.2035344	b

1it [1:22:52, 4972.95s/it]

0:	learn: 8.2488775	test: 8.2490712	best: 8.2490712 (0)	total: 961ms	remaining: 1h 20m 3s
500:	learn: 0.3658786	test: 0.4163518	best: 0.4163518 (500)	total: 8m 4s	remaining: 1h 12m 27s
1000:	learn: 0.2443495	test: 0.3134244	best: 0.3134244 (1000)	total: 16m 4s	remaining: 1h 4m 11s
1500:	learn: 0.1899888	test: 0.2721328	best: 0.2721328 (1500)	total: 24m 11s	remaining: 56m 24s
2000:	learn: 0.1563159	test: 0.2487276	best: 0.2487276 (2000)	total: 32m 15s	remaining: 48m 20s
2500:	learn: 0.1344752	test: 0.2349496	best: 0.2349496 (2500)	total: 40m 16s	remaining: 40m 14s
3000:	learn: 0.1176890	test: 0.2250708	best: 0.2250708 (3000)	total: 48m 19s	remaining: 32m 11s
3500:	learn: 0.1048885	test: 0.2181396	best: 0.2181396 (3500)	total: 56m 26s	remaining: 24m 9s
4000:	learn: 0.0946414	test: 0.2129739	best: 0.2129739 (4000)	total: 1h 4m 31s	remaining: 16m 6s
4500:	learn: 0.0859727	test: 0.2088643	best: 0.2088640 (4499)	total: 1h 12m 31s	remaining: 8m 2s
4999:	learn: 0.0787483	test: 0.2056625	best: 

2it [2:43:43, 4900.76s/it]

0:	learn: 8.2482436	test: 8.2577249	best: 8.2577249 (0)	total: 980ms	remaining: 1h 21m 40s
500:	learn: 0.3631944	test: 0.4111537	best: 0.4111537 (500)	total: 8m 6s	remaining: 1h 12m 48s
1000:	learn: 0.2428485	test: 0.3094263	best: 0.3094263 (1000)	total: 16m 9s	remaining: 1h 4m 33s
1500:	learn: 0.1876674	test: 0.2679032	best: 0.2679032 (1500)	total: 24m 15s	remaining: 56m 33s
2000:	learn: 0.1550493	test: 0.2450742	best: 0.2450742 (2000)	total: 35m 42s	remaining: 53m 30s
2500:	learn: 0.1326120	test: 0.2311052	best: 0.2311052 (2500)	total: 44m 2s	remaining: 44m
3000:	learn: 0.1161073	test: 0.2213452	best: 0.2213452 (3000)	total: 52m 42s	remaining: 35m 6s
3500:	learn: 0.1032874	test: 0.2144821	best: 0.2144821 (3500)	total: 1h 2m 50s	remaining: 26m 54s
4000:	learn: 0.0931706	test: 0.2093521	best: 0.2093521 (4000)	total: 1h 11m 38s	remaining: 17m 53s
4500:	learn: 0.0847915	test: 0.2052934	best: 0.2052934 (4500)	total: 1h 20m 14s	remaining: 8m 53s
4999:	learn: 0.0777016	test: 0.2020920	best:

3it [4:14:03, 5137.91s/it]

0:	learn: 8.2487808	test: 8.2515186	best: 8.2515186 (0)	total: 1.08s	remaining: 1h 30m 25s
500:	learn: 0.3675190	test: 0.4147237	best: 0.4147237 (500)	total: 10m 9s	remaining: 1h 31m 10s
1000:	learn: 0.2468538	test: 0.3142985	best: 0.3142985 (1000)	total: 19m 33s	remaining: 1h 18m 7s
1500:	learn: 0.1916118	test: 0.2725326	best: 0.2725326 (1500)	total: 27m 39s	remaining: 1h 4m 27s
2000:	learn: 0.1585755	test: 0.2496414	best: 0.2496414 (2000)	total: 36m	remaining: 53m 58s
2500:	learn: 0.1359779	test: 0.2354735	best: 0.2354735 (2500)	total: 44m 8s	remaining: 44m 6s
